In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os
sys.path.append('..')

import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV

from get_data import get_data, get_splitted_data, proteins, _get_raw_data
from models.active_model import ActiveLearningExperiment
from models.strategy import *
from models.utils import ObstructedY
from experiments.utils import *

from misc.config import c

In [ ]:
from sklearn.metrics import auc

In [ ]:
from get_data import _generate_fold_indices

In [ ]:
from experiments.experiment_runner import run_experiment
from experiments import fit_grid
import numpy as np

protein = '5ht6'
fingerprint = "ExtFP"
seed = 666
warm_start_percentage = 0.05
batch_size = 20
param_grid = {'h': [100]}

loader = ["get_splitted_data", {
    "seed": seed,
    "valid_size": 0.15,
    "n_folds": 2,
    'percent': 0.5}]

strategy = ('random_query', {})
preprocess_fncs = []

randomNB_exp = run_experiment("fit_grid",
                              recalculate_experiments=False,
                              n_jobs=4,
                              experiment_detailed_name="fit_randomNB_%s_%s_%s" % (strategy[0], protein, fingerprint),
                              base_experiment="fit_active_learning",
                              seed=seed,
                              grid_params=strategy[1],
                              base_experiment_kwargs={"strategy": strategy[0],
                                                      "preprocess_fncs": preprocess_fncs,
                                                      "protein": protein,
                                                      "fingerprint": fingerprint,
                                                      "warm_start_percentage": warm_start_percentage,
                                                      "batch_size": batch_size,
                                                      "base_model": "RandomNB",
                                                      "loader_function": loader[0],
                                                      "loader_args": loader[1],
                                                      "param_grid": param_grid})


In [ ]:
best = get_best(randomNB_exp.experiments, 'mean_wac_score')

In [ ]:
for k, v in best.results.iteritems():
    if 'auc_mean_wac' in k:
        print k, v

In [ ]:
for k, v in best.results.iteritems():
    if 'auc_wac' in k:
        print k, v

In [ ]:
for protein in proteins:
    _, y = _get_raw_data(protein, "ExtFP")
    print protein, y.shape[0]

In [ ]:

loader = ["get_splitted_data_clusterwise", {
        "seed": 666,
        "valid_size": 0.15,
        "n_folds": 1}]
preprocess_fncs = [["to_binary", {"all_below": True}]]
comps = [(p, 'ExtFP') for p in proteins]
    
data = get_data(comps, loader, preprocess_fncs)

In [ ]:
for key, value in data.iteritems():
    print key
    fold = value[0][0]
    print "train data", fold['X_train']['data'].shape[0]
    print "valid data", fold['X_valid']['data'].shape[0]
    print "A", len(fold['X_train']['cluster_A'])
    print "B", len(fold['X_train']['cluster_B'])
    print 
